In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
import torch, os, re , pandas as  pd, json
from sklearn.model_selection import train_test_split
from transformers  import DataCollatorForLanguageModeling, DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments,AutoConfig,TFGPT2LMHeadModel, TFTrainer
from datasets import Dataset

In [ ]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu"  
device = torch.device(dev)  

In [ ]:
device

MODEL AND TOKENIZER LOADING

In [ ]:
base_model=GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
base_model.num_parameters

In [ ]:
base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

DATA PREPARATION

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
original_path = '/content/gdrive/My Drive/Datasets/'

In [ ]:
data_dir = os.path.join(original_path)
data_dir

In [ ]:
df_url=os.path.join(data_dir, 'big_data.json')

In [ ]:
data=pd.read_json(df_url)

In [ ]:
col_name='element'

In [ ]:
dic={
     'element':[],
     
     'address':[]
}
df=pd.DataFrame(dic)
df


In [ ]:
for k,i in enumerate(data.rows.copy()):
  df.loc[k]=[i['element'], i['element']['address']]

In [ ]:
for i in df['element']:
    del i['address']

In [ ]:
df.head()

In [ ]:
attributes=[]
for i in range(df.shape[0]):
    attributes.extend( df.iloc[:][col_name][i].keys())

In [ ]:
attr_mean={}
for el in set(attributes):
   if attributes.count(el) >= 100:
      attr_mean[el]=attributes.count(el)

In [ ]:
len(attr_mean.keys())

In [ ]:
empt_arr={}

for key in list(attr_mean.keys()):
  empt_arr[key]=''
 

In [ ]:
df_edir=df.copy()
for i in range(df.shape[0]): 
        cont=df_edir.iloc[i][col_name].copy()
        for key in df_edir.iloc[i][col_name].keys():
         
         if key not in list(attr_mean.keys()):
            del cont[key]
         
         empt_arr2=empt_arr.copy()
         empt_arr2.update(cont)
         df_edir[col_name][i]=empt_arr2
     

In [ ]:
df_edir.head()

In [ ]:
for k,i in enumerate(data.rows.copy()):
  df_edir['element'][k]=" ".join(sorted(i['element'].values()))

In [ ]:
df_edir.head()

In [ ]:
df_text_ids=[]

TOKENIZATION

In [ ]:
# the eos and bos tokens are defined
bos = '<|endoftext|>'
eos = '<|EOS|>'
pad = '<|pad|>'

In [ ]:
special_tokends_dict={'eos_token':eos,
                      'bos_token':bos,
                      'pad_token':pad}

In [ ]:
#the new token is added to the tokenizer
num_added_toks=base_tokenizer.add_special_tokens(special_tokends_dict)

In [ ]:
config=AutoConfig.from_pretrained('gpt2',
                                  bos_token_id=base_tokenizer.bos_token_id,
                                  eos_token_id=base_tokenizer.eos_token_id,
                                  
                                  pad_token_id=base_tokenizer.pad_token_id,
                                    output_hidden_states=False)

In [ ]:
base_model=TFGPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
base_model.resize_token_embeddings(len(base_tokenizer))


In [ ]:
dd_toks=df_edir.copy()

In [ ]:
for k,i in enumerate(data.rows.copy()):
    df_edir['element'][k]=bos+' ' +df_edir['element'][k]#+' '+eos

In [ ]:
for k,i in enumerate(data.rows.copy()):
    df_edir['address'][k]=bos+' ' +df_edir['address'][k]#+' '+eos

In [ ]:
df_edir.head()

In [ ]:
df_text_ids=[]
df_addr_ids=[]

In [ ]:
for k,i in enumerate(data.rows.copy()):
  df_text_ids.append((base_tokenizer.encode( df_edir['element'][k])))

In [ ]:
for k,i in enumerate(data.rows.copy()):
  df_addr_ids.append((base_tokenizer.encode( df_edir['address'][k])))

In [ ]:
import tensorflow as tf

In [ ]:
df_edir['text_ids']=df_text_ids

In [ ]:
df_edir['addr_ids']=df_addr_ids

In [ ]:
df_edir['addr_ids'].head()

In [ ]:
df_edir['text_ids'].head()

In [ ]:
size=1000

In [ ]:
import numpy as np

In [ ]:
for i in range(len(df_edir['text_ids'])):
   df_edir['text_ids'][i]=np.array(df_edir['text_ids'][i])

In [ ]:
for i in range(len(df_edir['addr_ids'])):
   df_edir['addr_ids'][i]=np.array(df_edir['addr_ids'][i])

In [ ]:
for i in range(len(df_edir['text_ids'])):
  df_edir['text_ids'][i]=(df_edir['text_ids'][i]).astype(np.float32)

In [ ]:
train_data=tf.keras.preprocessing.sequence.pad_sequences(list(df_edir['text_ids'][:size].values),padding='post')

In [ ]:
  import numpy as np


In [ ]:
def add_eos(data):
  data=tf.keras.preprocessing.sequence.pad_sequences(data,padding='post')
  new_data=[]
  for i in range(len(data)):
    new_data.append(np.append(data[i],50257))
  return new_data

In [ ]:
train_data =add_eos(train_data)

In [ ]:
test_daata =add_eos(test_data)

In [ ]:
test_daata[0]

In [ ]:
test_data=tf.keras.preprocessing.sequence.pad_sequences(list(df_edir['text_ids'][size:size+700].values),padding='post')

In [ ]:
df_edir.head()

In [ ]:
BATCH_SIZE_PER_REPLICA = 28
EPOCHS = 6
INITAL_LEARNING_RATE = 0.001
try:
    BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
except NameError as e:
    BATCH_SIZE = BATCH_SIZE_PER_REPLICA
BUFFER_SIZE = len(df_token)

# prepare data for consumption
train_ds = (
   df_token.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
)
test_ds =df_token.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print("Running on TPU ", tpu.cluster_spec().as_dict()["worker"])
except ValueError:
    tpu = None
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
# Drecreasing learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    INITAL_LEARNING_RATE,
    decay_steps=500,
    decay_rate=0.7,
    staircase=True)

# initialize model, use_cache=False important! else wrong shape at loss calc
with strategy.scope():
    model = TFGPT2LMHeadModel.from_pretrained(
        "gpt2",
        use_cache=False,
        pad_token_id=base_tokenizer.pad_token_id,
        eos_token_id=base_tokenizer.eos_token_id,
    )
    model.resize_token_embeddings(len(base_tokenizer))
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(optimizer=optimizer, loss=model.compute_loss)
    model.summary()

In [ ]:
train = tf.data.Dataset.from_tensor_slices(
    (
        {"input_ids": train_data,
        }
    )
)


In [ ]:
test = tf.data.Dataset.from_tensor_slices(
    (
        {"input_ids": test_daata,
        }
    )
)


In [ ]:
hist = model.fit(
    X=train_data,
    y
    validation_data=test_daata,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1,
)

In [ ]:
class AddreesGeneration(Dataset):
  def __init__(self, txt_list,label_list,tokenizer, max_length):
            #define variables
            self.input_ids=[]
            self.attn_mask=[]
            self.labels=[]
            map_label=[]
            #iterate through the dataset
            for txt, label in zip(txt_list,label_list):
                  #prepare the text
                  prep_text=f'<startoftext>Element: {txt}\n Address: {label}<endoftext>'
                  #tokenize
                  encodings_dict=tokenizer(prep_text,truncation=True,max_length=max_length,
                                           padding='max_length')
                  
                  #append to list
                  self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
                  self.attn_mask.append(torch.tensor(encodings_dict['attention_mask']))
                  self.labels.append(label)

  def __len__(self):
      return len(self.input_ids)

  
  def __getiten__(self,idx):
      return self.input_ids[idx], self.attn_mask[idx], self.labels[id]

LOAD MODEL AND DATA

set model name

In [ ]:
model_name="gpt2"

SEED

In [ ]:
torch.manual_seed(42)

In [ ]:
from transformers import GPT2TokenizerFast

LOAD TOKENIZER AND MODEL

In [ ]:
tokenizer=GPT2TokenizerFast.from_pretrained(model_name,bos_token='<startoftext>',eos_token='<endoftext>',pad_token='<pad>')

In [ ]:
model=GPT2LMHeadModel.from_pretrained(model_name).cuda()

In [ ]:
model.resize_token_embeddings(len(tokenizer))

In [ ]:
df_edir.head()

In [ ]:
size=1000

In [ ]:
train_dataset=df_edir[:size]

In [ ]:
test_dataset=df_edir[size:size+700]

In [ ]:
size=1000

In [ ]:
train_data=AddreesGeneration(txt_list=train_dataset['element'],label_list=train_dataset['address'],tokenizer=tokenizer, max_length=300)

In [ ]:
test_data=AddreesGeneration(txt_list=test_dataset['element'],label_list=test_dataset['address'],tokenizer=tokenizer, max_length=300)

TRAIN

CREATING TRAINING ARGUMENTS

In [ ]:
training_args=TrainingArguments(output_dir='results', num_train_epochs=2, logging_steps=10,
                               save_strategy='epoch',
                                per_device_train_batch_size=2,per_device_eval_batch_size=2,
                                warmup_steps=100,weight_decay=0.01,logging_dir='logs')

In [ ]:
df_train=pd.DataFrame(
    {
         'input_ids':train_data.input_ids,
          'attention_mask':train_data.attn_mask,
          'labels':train_data.labels
    }
)

In [ ]:
df_train.head()

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling


In [ ]:
df_edir.head()

In [ ]:
import numpy as np

In [ ]:
np.savetxt('train_data.txt',df_edir.values,fmt='%s')

START TRAINING

In [ ]:
Trainer(model=model,args=training_args,train_dataset=df_train[['input_ids','attention_mask']]).train()

In [ ]:
data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )

In [ ]:
trainer.train()